# Diagnóstico de Diabetes con XGBoost
En el cuaderno anterior, hablábamos de _XGBoost_, su potencia y flexibilidad. Lo seguiremos usando para hacer otra clasificación binaria. Nuestro objetivo es predecir si una persona tiene diabetes en función de varios indicadores médicos. ¿Por qué _XGBoost_? Porque combina la fuerza de los árboles con técnicas de _boosting_, logrando modelos muy precisos y a la vez interpretables en importancia de variables. ¿Tendremos buenos resultados como en los modelos anteriores? Comenzamos con los datos, que son reales y obtuve [aquí, en Kaggle](https://www.kaggle.com/datasets/mathchi/diabetes-data-set/data).

## Carga y exploración de los datos
Primero, descargamos el archivo `diabetes.csv` desde _Kaggle_ y lo cargamos en `pandas`. Vamos a conocerlo y limpiarlo.

In [1]:
# Usaremos pandas, como siempre.
import pandas as pd

df = pd.read_csv('diabetes.csv')
pd.set_option('future.no_silent_downcasting', True)

In [2]:
# Impresión e información.
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


Acabo de sonreír porque no hay datos nulos. ¿Pero habrá duplicados?

In [3]:
print(df.duplicated().sum())

0


Tampoco hay duplicados. Analicemos el _DataFrame_ a nivel estadístico.

In [4]:
print(df.describe())

       Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
count   768.000000  768.000000     768.000000     768.000000  768.000000   
mean      3.845052  120.894531      69.105469      20.536458   79.799479   
std       3.369578   31.972618      19.355807      15.952218  115.244002   
min       0.000000    0.000000       0.000000       0.000000    0.000000   
25%       1.000000   99.000000      62.000000       0.000000    0.000000   
50%       3.000000  117.000000      72.000000      23.000000   30.500000   
75%       6.000000  140.250000      80.000000      32.000000  127.250000   
max      17.000000  199.000000     122.000000      99.000000  846.000000   

              BMI  DiabetesPedigreeFunction         Age     Outcome  
count  768.000000                768.000000  768.000000  768.000000  
mean    31.992578                  0.471876   33.240885    0.348958  
std      7.884160                  0.331329   11.760232    0.476951  
min      0.000000                  

## Información importante
Así de fácil ya tenemos una idea de la edad de las personas, su índice de masa corporal, presión sanguínea, embarazos, etc. Pero hay más cosas que debes saber. Este _data set_ es de la _National Institute of Diabetes and Digestive and Kidney Diseases_. Todas las pacientes son mujeres de por lo menos 21 años de origen indio. Voy a pasar los nombres de las columnas a lo que nos dice la convención _snake_case_ y reemplazar 0 en columnas que no deberían tenerlos. Así habremos terminado la limpieza de nuestro _data set_.

In [5]:
# Primero pasemos las columnas a minúsculas.
df.columns = df.columns.str.lower()

# Y modifiquemos directamente los nombres de algunas columnas.
df = df.rename(columns = {
    'bloodpressure': 'blood_pressure',
    'skinthickness': 'skin_thickness',
    'diabetespedigreefunction': 'diabetes_pedigree_function'
})

# Comprobación.
print(df.columns)

Index(['pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 'insulin',
       'bmi', 'diabetes_pedigree_function', 'age', 'outcome'],
      dtype='object')


In [6]:
# Reemplazamos 0 en columnas que no pueden ser 0.
for col in ['glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi']:
    df[col] = df[col].replace(0, pd.NA)
    df[col] = df[col].fillna(df[col].median())

Estamos listos.

## División en entrenamiento y prueba
Como sabes, al hacer modelos predictivos, la información se divide en datos de entrenamiento y datos de prueba. El modelo se entrena con los datos de entrenamiento, el 80 por ciento del _data set_, y hace predicciones con datos de prueba que son los que no conoce, ese 20 por ciento restante. Así podemos comprobar la efectividad del modelo con datos que no han influido en su entrenamiento.

In [7]:
# Con este módulo podemos crear las variables de entrenamiento y prueba. 
from sklearn.model_selection import train_test_split

# 'X' almacena las características y 'y' almacena la variable objetivo.
X = df.drop('outcome', axis=1)
y = df['outcome']

# Así se declaran las variables.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Escalado de variables
Normalizaremos los datos con `StandarScaler`. Los modelos basados en árboles, como _XGBoost_, son insensibles a las diferentes escalas de las variables, ya que funcionan dividiendo datos en umbrales y no dependen de la distancia o magnitud de las características.La normalización suele ser más relevante para modelos que dependen de medidas de distancia como regresión logística, _SVM_ o redes neuronales, pero nada pasa si lo hacemos.

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 6. Entrenamiento con XGBoost
Ahora sí, entrenamos con _XGBoost_:

In [9]:
import xgboost as xgb

model = xgb.XGBClassifier(eval_metric='logloss', n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

## Predicciones y evaluación
Vamos a evaluar si el modelo ha aprendido y cuál es su precisión, entre otras métricas.

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test_scaled)

print("Precisión:", accuracy_score(y_test, y_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("Reporte:\n", classification_report(y_test, y_pred))

Precisión: 0.7142857142857143
Matriz de confusión:
 [[75 24]
 [20 35]]
Reporte:
               precision    recall  f1-score   support

           0       0.79      0.76      0.77        99
           1       0.59      0.64      0.61        55

    accuracy                           0.71       154
   macro avg       0.69      0.70      0.69       154
weighted avg       0.72      0.71      0.72       154



## Analisis de las métricas

- La precisión general es de 0.79, es decir, el modelo acierta en el 79% de los casos.
- La matriz de confusión indica 75 verdaderos negativos (predijo bien "no diabetes"), 35 verdaderos positivos (predijo bien "diabetes"), 24 falsos positivos (predijo diabetes donde no había), 20 falsos negativos (no detectó diabetes donde sí había).
- El reporte de clasificación nos dice que para la clase 0 (no diabetes) tiene mejor precisión (0.79) y buen _recall_ (0.76), para la clase 1 (diabetes) tiene 0.59 de precisión y _recall_ de 0.64, es decir, el modelo tiene más dificultades para identificar correctamente todos los casos de diabetes. El F1-score bajo en la clase 1 indica que el modelo podría estar sesgado a predecir más casos de no diabetes.

## Conclusión
El modelo es mejor prediciendo la ausencia de diabetes que la presencia de la enfermedad. Podríamos ajustar parámetros, balancear el _dataset_ o explorar técnicas de _oversampling_ para mejorar la detección de positivos. También podríamos afinar el enfoque del modelo con base en la importancia de las variables. El modelo tiene margen de mejora. Algunos de los conceptos de esta conclusión serán desarrollados en los siguientes cuadernos. Ya sabes, _stay tuned!_